In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications import DenseNet121
from keras.models import Model
from keras.layers import Input
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt
import os

Re-build VGG16 with output is layer fc2

In [ ]:
model_vgg16 = VGG16()
input = Input(shape=(224,224,3))
output = model_vgg16.get_layer('fc2').output
model_vgg16_fc2 = Model(model_vgg16.input, output)
model_vgg16_fc2.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

Extract feature function

In [ ]:
def extract_feature(img_path):
  img = image.load_img(img_path, target_size=(224,224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = preprocess_input(img)
  imgs.append(img)
  feature = model_vgg16_fc2.predict(img)
  return feature

In [ ]:
Calculate AP function

In [ ]:
def AP(actual,query_res,total):
  count = 1
  ap = 0
  for i in range (len(query_res)):
    if query_res[i] in actual:
        ap += count/(i+1)
        count+=1
  return ap/total

In [ ]:
def generate_ground_truth(gt_path,size=4):
  grounds = os.listdir(gt_path)
  ground_set = dict()
  ground_answer = dict()
  for ground in grounds:
    with open(os.path.join(gt_path,ground),'r') as f:
      ground = ground.split('.')[0]
      data = [x for x in f.read().split('\n') if x]
      # answer list
      ground_answer[ground] = data
      rd = np.arange(len(data))
      np.random.shuffle(rd)
      rd = rd[:size]
      data = np.array(data)
      # test_list
      ground_set[ground] = data.take(rd).tolist()
      del f
  return ground_set,ground_answer
test_set,answer_set = generate_ground_truth(gt_path='/content/drive/My Drive/CS336_data/groundtruth', size=4)
print(test_set)
print(answer_set)

{'BuuDienThanhPho': ['BuuDienThanhPho6.jpg', 'BuuDienThanhPho4.jpg', 'BuuDienThanhPho8.jpg', 'BuuDienThanhPho1.jpg'], 'NhaThoDucBa': ['NhaThoDucBa5.jpg', 'NhaThoDucBa7.jpg', 'NhaThoDucBa6.jpg', 'NhaThoDucBa2.jpg'], 'VinhCamRanh': ['VinhCamRanh2.jpg', 'VinhCamRanh8.jpg', 'VinhCamRanh6.jpg', 'VinhCamRanh7.jpg'], 'QuocHocHue': ['QuocHocHue4.jpg', 'QuocHocHue6.jpg', 'QuocHocHue2.jpg', 'QuocHocHue8.jpg']}
{'BuuDienThanhPho': ['BuuDienThanhPho1.jpg', 'BuuDienThanhPho2.jpg', 'BuuDienThanhPho3.jpg', 'BuuDienThanhPho4.jpg', 'BuuDienThanhPho5.jpg', 'BuuDienThanhPho6.jpg', 'BuuDienThanhPho7.jpg', 'BuuDienThanhPho8.jpg'], 'NhaThoDucBa': ['NhaThoDucBa1.jpg', 'NhaThoDucBa2.jpg', 'NhaThoDucBa3.jpg', 'NhaThoDucBa4.jpg', 'NhaThoDucBa5.jpg', 'NhaThoDucBa6.jpg', 'NhaThoDucBa7.jpg', 'NhaThoDucBa8.jpg'], 'VinhCamRanh': ['VinhCamRanh1.jpg', 'VinhCamRanh2.jpg', 'VinhCamRanh3.jpg', 'VinhCamRanh4.jpg', 'VinhCamRanh5.jpg', 'VinhCamRanh6.jpg', 'VinhCamRanh7.jpg', 'VinhCamRanh8.jpg'], 'QuocHocHue': ['QuocHocHue1.

In [ ]:
def generate_groundtruth

Viết file json chứa features của từng ảnh

In [ ]:
import json
import numpy as np
feature_data = {}
folder_path = "/content/drive/My Drive/CS336_data/image"
imgs = os.listdir(folder_path)
for img in imgs[:]:
  path = folder_path +'/'+ img 
  feature = extract_feature(path)
  feature_data[img] = np.array(feature).tolist()

In [ ]:
with open('feature_data.json', 'w') as dt:
  json.dump(feature_data, dt)
print(len(feature_data))

300


Tải file feature_data.json về và sử dụng sau này

In [ ]:
with open("/content/drive/My Drive/CS336_data/feature_data.json", "r") as fin:
    feature_data = json.load(fin)

In [ ]:
print(len(feature_data))

300


Calculate distance function

In [ ]:
def cal_distance(feature_query, feature_destination, function):
  
  feature_query=np.array(feature_query[0])
  feature_destination = np.array(feature_destination[0])

  if function=='L2':
    return np.linalg.norm(feature_query-feature_destination)
  if function=='COS':
    return np.dot(feature_query,feature_destination)/(np.linalg.norm(feature_query)*np.linalg.norm(feature_destination))
  if function=='DOT':
    return np.dot(feature_query, feature_destination)
    

In [ ]:
a= np.array([1,2,3])
b= np.array([4,5,6])
print(np.dot(a,b))
print(a.shape)

32
(3,)


Query image:

In [ ]:
def query_image(query_img_path, feature_data, function):
  query_img = image.load_img(query_img_path, target_size=(224,224))
  query_img = image.img_to_array(query_img)
  query_img = np.expand_dims(query_img, axis=0)
  query_img = preprocess_input(query_img)
  feature_query = model_vgg16_fc2.predict(query_img)
  
  query_res = {}

  for feature in feature_data:
    query_res[feature] = cal_distance(feature_query, feature_data[feature], function)
  if function=='L2':
    query_res_sort = sorted(query_res.items(), key=lambda x:x[1], reverse=False)
  else:
    query_res_sort = sorted(query_res.items(), key=lambda x:x[1], reverse=True)
  return query_res_sort[:10]

Final

In [ ]:
def mAP(function):
  APs = []
  number_of_queries = 10
  expected_correct = 8
  locations = list(answer_set.keys())
  
  print('###################### USING {} FUNCTION ######################'.format(function))
  print('\n\n\n')
  final_map = 0
  for location in locations:
    #print(location)
    #print(test_set[location])
    sum_pre = 0
    for _ in test_set[location]:
      query_res = query_image(query_img_path=str('/content/drive/My Drive/CS336_data/image/'+_), feature_data=feature_data, function=function)
      #print(query_res)
      tmp = []
      for __ in query_res:
        tmp.append(__[0])
      print("Result of query {} is: ".format(_),tmp)
        #print(answer_set[location])
      print('AP of this result query is: ',AP(actual=answer_set[location], query_res=tmp, total=expected_correct))
      sum_pre += AP(actual=answer_set[location], query_res=tmp, total=expected_correct)
    print('=> mAP of this is: ',sum_pre/4)
    final_map+=sum_pre/4
    print('----------------------------')
  print('=====================================================================================')
  print('mAP of this query is: ',final_map/4)


In [ ]:
mAP('L2')

###################### USING L2 FUNCTION ######################




Result of query BuuDienThanhPho6.jpg is:  ['BuuDienThanhPho6.jpg', 'NhaThoDucBa7.jpg', 'NhaThoDucBa3.jpg', 'NhaThoDucBa1.jpg', 'BuuDienThanhPho8.jpg', 'QuocHocHue4.jpg', 'BuuDienThanhPho2.jpg', 'image_2057_th.png', 'NhaThoDucBa5.jpg', 'image_7201_th.png']
AP of this result query is:  0.22857142857142856
Result of query BuuDienThanhPho4.jpg is:  ['BuuDienThanhPho4.jpg', 'BuuDienThanhPho2.jpg', 'BuuDienThanhPho1.jpg', 'BuuDienThanhPho5.jpg', 'BuuDienThanhPho7.jpg', 'NhaThoDucBa7.jpg', 'BuuDienThanhPho3.jpg', 'NhaThoDucBa3.jpg', 'QuocHocHue7.jpg', 'image_6979_th.png']
AP of this result query is:  0.7321428571428571
Result of query BuuDienThanhPho8.jpg is:  ['BuuDienThanhPho8.jpg', 'BuuDienThanhPho2.jpg', 'BuuDienThanhPho5.jpg', 'BuuDienThanhPho7.jpg', 'BuuDienThanhPho4.jpg', 'NhaThoDucBa1.jpg', 'BuuDienThanhPho1.jpg', 'image_2678_th.png', 'BuuDienThanhPho6.jpg', 'NhaThoDucBa3.jpg']
AP of this result query is:  0.829365079

In [ ]:
mAP('DOT')

###################### USING DOT FUNCTION ######################




Result of query BuuDienThanhPho6.jpg is:  ['BuuDienThanhPho6.jpg', 'BuuDienThanhPho8.jpg', 'image_750_th.png', 'NhaThoDucBa5.jpg', 'QuocHocHue1.jpg', 'image_4443_th.png', 'BuuDienThanhPho7.jpg', 'BuuDienThanhPho2.jpg', 'image_797_th.png', 'NhaThoDucBa8.jpg']
AP of this result query is:  0.36607142857142855
Result of query BuuDienThanhPho4.jpg is:  ['BuuDienThanhPho4.jpg', 'BuuDienThanhPho7.jpg', 'BuuDienThanhPho2.jpg', 'QuocHocHue1.jpg', 'BuuDienThanhPho1.jpg', 'BuuDienThanhPho5.jpg', 'BuuDienThanhPho3.jpg', 'image_4971_th.png', 'NhaThoDucBa2.jpg', 'QuocHocHue7.jpg']
AP of this result query is:  0.6863095238095237
Result of query BuuDienThanhPho8.jpg is:  ['BuuDienThanhPho8.jpg', 'BuuDienThanhPho7.jpg', 'BuuDienThanhPho2.jpg', 'QuocHocHue1.jpg', 'NhaThoDucBa8.jpg', 'BuuDienThanhPho4.jpg', 'BuuDienThanhPho1.jpg', 'BuuDienThanhPho5.jpg', 'image_4971_th.png', 'image_750_th.png']
AP of this result query is:  0.64136904761

In [ ]:
mAP('COS')

###################### USING COS FUNCTION ######################




Result of query BuuDienThanhPho6.jpg is:  ['BuuDienThanhPho6.jpg', 'BuuDienThanhPho8.jpg', 'NhaThoDucBa5.jpg', 'BuuDienThanhPho2.jpg', 'NhaThoDucBa3.jpg', 'NhaThoDucBa7.jpg', 'QuocHocHue7.jpg', 'image_2231_th.png', 'BuuDienThanhPho7.jpg', 'BuuDienThanhPho1.jpg']
AP of this result query is:  0.4618055555555556
Result of query BuuDienThanhPho4.jpg is:  ['BuuDienThanhPho4.jpg', 'BuuDienThanhPho2.jpg', 'BuuDienThanhPho7.jpg', 'BuuDienThanhPho1.jpg', 'BuuDienThanhPho5.jpg', 'BuuDienThanhPho3.jpg', 'QuocHocHue1.jpg', 'NhaThoDucBa7.jpg', 'QuocHocHue7.jpg', 'NhaThoDucBa2.jpg']
AP of this result query is:  0.75
Result of query BuuDienThanhPho8.jpg is:  ['BuuDienThanhPho8.jpg', 'BuuDienThanhPho2.jpg', 'BuuDienThanhPho7.jpg', 'BuuDienThanhPho5.jpg', 'BuuDienThanhPho4.jpg', 'BuuDienThanhPho1.jpg', 'BuuDienThanhPho6.jpg', 'NhaThoDucBa8.jpg', 'QuocHocHue7.jpg', 'BuuDienThanhPho3.jpg']
AP of this result query is:  0.975
Result of qu

Như kết quả ta vừa tính đuược ta thấy hàm cosin tính ra kết quả chính xác cao nhất vì mục đích của chúng ta là tìm ra đúng mục tiêu chọn lựa chứ không phải là theo mức độ liên quan